---
title: Downloading Climate Data from ERA5
short_title: ERA5 Climate Data
---

In this notebook we will demonstrate how we can use Climate Tools and `dhis2eo` to retrieve climate data for a set of DHIS2 organisation units, based on the [ERA5 dataset](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics?tab=download), hosted at the [ECMWF Climate Data Store (CDS)](https://cds.climate.copernicus.eu/datasets). 

----------------------------------------
## Requirements 

### 1. Create an ECMWF User

Before you can download the dataset programmatically, you need to [create an ECMWF user](https://www.ecmwf.int/user/login).

### 2. Authenticate with your ECMWF user

Next, you need to authenticate with this user based on your user credentials:

- Go to the [CDSAPI Setup page](https://cds.climate.copernicus.eu/how-to-api) and make sure to login.
- Once logged in, scroll down to the section "Setup the CDS API personal access token". 
  - This should show your login credentials, and look something like this:

        url: https://cds.climate.copernicus.eu/api
        key: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

- Copy those two lines to a file `.cdsapirc` in your user's $HOME directory.

### 3. Accept the dataset license

Lastly, ECMWF requires that you manually accept the user license for each dataset that you download. 

- Start by visiting the Download page of the dataset we are interested in: ["ERA5 post-processed daily statistics on single levels from 1940 to present"](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics?tab=download). 
- Scroll down until you get to the "Terms of Use" section.
- Click the button to accept and login with your user if you haven't already. 

--------------------------------------------
## Step-by-step Workflow

We start by importing the necessary libraries:

In [2]:
import dhis2eo
import dhis2eo.org_units
import dhis2eo.data.cds

### Step 1. Connect to DHIS2

The first thing to do is connect to an instance of DHIS2. For our example, we will connect to a local instance of DHIS2 containing the standard Sierra Leone demo database, but you should be able to switch out the instance url and credentials to work directly with your own database. 

In [ ]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Create DHIS2 client connection
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district"
)
client = DHIS2Client(settings=cfg)

# Verify connection
info = client.get_system_info()
print("Current DHIS2 version:", info["version"])

Current DHIS2 version: 2.42.3


### Step 2: Retrieve organisation units

Next, we get the organisation units from our DHIS2 instance and load them into a format we can work with: 

In [5]:
org_units_geojson = client.get_org_units_geojson(level=2)
org_units = dhis2eo.org_units.from_dhis2_geojson(org_units_geojson)
org_units

,org_unit_id,name,level,geometry
0,O6uvpzGd5pu,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


### Step 3: Download data for multiple months

The `dhis2eo.data` module provides a convenience function for downloading daily climate data from the [ERA5 post-processed daily statistics on single levels from 1940 to present](https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics). This function downloads the most commonly requested ERA5 climate variables: 2m temperature and total precipitation. Since data downloads from the Climate Data Store (CDS) can be slow, the function also caches the download results and reuses it if the file has already been downloaded. 

Since the data can only be downloaded one month at a time, downloading data for a range of months can be done by looping through the years and months we are interested in. Inside the loop, we provide the year, month, and org_units we want to download for. The region to download data for is automatically calculated from the provided organisation units. 

For this notebook, let's download data for the last 3 months: 

In [10]:
start_year = 2025
start_month = 7
end_year = 2025
end_month = 9

for year in range(start_year, end_year+1):
    for month in range(1, 12+1):
        
        # skip months before or after our defined time range
        if (year,month) < (start_year,start_month):
            continue
        if (year,month) > (end_year,end_month):
            continue

        # download the climate data
        # commented out for this notebook
        print(f'Month: {year}-{month}')
        data = dhis2eo.data.cds.get_daily_era5_data(year, month, org_units)

        # do something with the data
        # e.g. save to disk, aggregate, or import to DHIS2
        # ...

Month: 2025-7
dhis2eo.data.cds - INFO - Loading from cache: /tmp/cds_daily-era5_params-ca5bab_region-37098a_2025-07.nc
Month: 2025-8
dhis2eo.data.cds - INFO - Loading from cache: /tmp/cds_daily-era5_params-ca5bab_region-37098a_2025-08.nc
Month: 2025-9
dhis2eo.data.cds - INFO - Loading from cache: /tmp/cds_daily-era5_params-ca5bab_region-37098a_2025-09.nc


To inspect the contents of the downloaded data, let's view the data of the last element in the loop: 

In [11]:
data.to_xarray()

<xarray.Dataset> Size: 40kB
Dimensions:     (valid_time: 29, latitude: 13, longitude: 13)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 232B 2025-09-01 ... 2025-09-29
  * latitude    (latitude) float64 104B 9.918 9.668 9.418 ... 7.418 7.168 6.917
  * longitude   (longitude) float64 104B -13.3 -13.05 -12.8 ... -10.55 -10.3
    number      int64 8B ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(29, 13, 13), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 20kB dask.array<chunksize=(29, 13, 13), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-11T11:39 GRIB to CDM+CF via cfgrib-0.9.1...

We see that this is data for the month of September 2025, and contains data variables `t2m` (2m temperature), and `tp` (total precipitation)

### Step 4: Process the data

The loop in the previous step only downloaded the data, but didn't actually do anything with it. For guidance on how to further process the downloaded data, see: 

* [Aggregate the data to your organisation units](../../aggregation/org-unit-aggregation.ipynb).
* [Import the aggregated data to DHIS2](../../import-data/using-python-client.ipynb).

## Next steps

This notebook has showed how to download and potentially process the ERA5 climate data. As these are daily updated data, this script should be run regularly to continue feeding DHIS2 with the latest data. This can be done by automatically triggering the script to be run at regular intervals, e.g. via a `cron` job (guide to be added in the future). 